# Packages 

In [21]:
import matplotlib as mp
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import glob
import sys
import os
from scipy.interpolate import interpolate
from scipy.interpolate import RectBivariateSpline
from scipy.interpolate import griddata


from spectral_analysis_functions import *
# Using OceanDataLab reader 
#from sentinel2_l1c import *
import numpy as np
from S2_read import *
from s2_angs import *
# bands=['B04','B08','B03','B02'];  



# Image Reading

In [22]:
XML_File = find('MTD_TL.xml', S2path)
XML_File_DS=find('MTD_DS.xml',S2path)
if XML_File is None:
    print("Le fichier 'MTD_TL.xml' n'a pas été trouvé dans le répertoire spécifié:", S2path)


[imgs,NX,NY,nx,ny,dx,dy,offspec,phitrig,thetav,loncenter,latcenter,detector,indexX,indexY]=S2_read(S2path,boxi,bands)

        

NameError: name 'S2path' is not defined

## image based on detectors

IndexX et indexY: pixels; 
array1: image as a function of detectors

In [23]:
print('lon,lat:',np.degrees(loncenter),np.degrees(latcenter))

nb=np.shape(bands)[0]


array1=np.squeeze(detector[1,:,:])
fig, ax = plt.subplots() 
im=ax.pcolor(indexX,indexY,array1)
fig.colorbar(im)
ax.set_title('detector index and FFT box location')
ax.set_xlabel('pixel index')
ax.set_ylabel('pixel index')
ax.axis('equal')
rect = mp.patches.Rectangle((boxi[0], boxi[2]), boxi[1]-boxi[0], boxi[3]-boxi[2], linewidth=2,
                         edgecolor='k', facecolor="none")

ax.add_patch(rect)
plt.show()

NameError: name 'loncenter' is not defined

In [25]:
loncenter=loncenter*180/(math.pi)
latcenter=latcenter*180/(math.pi)



NameError: name 'loncenter' is not defined

# Identifying the detector

In [26]:
def centre_carre(x0,xy0,taille_boite):
    centrex=x0+taille_boite/2
    centrey=y0+taille_boite/2
    return centrex, centrey

x0=boxi[0]
y0=boxi[2]
cote=boxi[1]-boxi[0]
centre=centre_carre(x0,y0,cote)

NameError: name 'boxi' is not defined

In [27]:
indexX2, indexY2 = np.meshgrid(indexX, indexY)
print('shape indexX',np.shape(indexX))
print('shape indexY',np.shape(indexY))
print('shape array1',np.shape(array1))


points = np.column_stack((indexX.ravel(), indexY.ravel()))
values = array1.ravel()

# Interpolation
value = griddata(points, values, centre, method='linear')
value = value.item()  
print(value)

NameError: name 'indexX' is not defined

### dictionary of detector numbers

In [28]:
mapping = {
    1: 'D01',
    2: 'D02',
    3: 'D03',
    4: 'D04',
    5: 'D05',
    6: 'D06',
    7: 'D07',
    8: 'D08',
    9: 'D09',
    10: 'D10',
    11: 'D11',
    12: 'D12'
}
if round(value) in mapping:
    Det = mapping[round(value)]
else:
    Det = "Valeur non trouvée"
print(Det)

detector_find=int(value)

NameError: name 'value' is not defined

### define the xml path and then use renaud's code

In [29]:
datastrip_dir = os.path.join(S2path, "DATASTRIP")

datastrip_contents = os.listdir(datastrip_dir)

directories = [item for item in datastrip_contents if os.path.isdir(os.path.join(datastrip_dir, item))]

if len(directories) == 1:
    xml_path = os.path.join(datastrip_dir, directories[0], "MTD_DS.xml")
    print(xml_path)
else:
    print("Il n'y a pas exactement un seul répertoire dans DATASTRIP.")

NameError: name 'S2path' is not defined

### Renaud Binet's Code

In [30]:
# Renaud Binet's Algo


dt1=!python main_computeDT.py -lon {loncenter} -lat {latcenter} -src B2 -dst B4 -det {Det} -sat {Sat} -eph {xml_path}
print('dt=',dt1[0])

dt= usage: main_computeDT.py [-h] -lon LONGITUDE -lat LATITUDE -src SRC_BAND -dst


# Image normalisation and filtering

### Normalisation
We look at the band B04 and B02 -->  bands=['B04','B08','B03','B02'];  

In [32]:
# normalizes the image values
img1=imgs[0,:,:]/np.median(imgs[0,:,:].flatten())
img2=imgs[3,:,:]/np.median(imgs[3,:,:].flatten())


NameError: name 'imgs' is not defined

# Fourier transform with two images

ntile: numbers of tile used for the welch. In the function FTT2D_two_arrays_nonan replaces the nan values in the images with the local mean


In [33]:

ntile=16 # le 10/01/25
# ntile=8
(Eta,Etb,ang,angstd,coh,crosr,phases,kx2,ky2,dkx,dky)=FFT2D_two_arrays(img1,img2,10,10,ntile,isplot=0)
print(np.shape(Eta))
print(np.shape(kx2))
print(np.shape(ky2))


phiblind1=phitrig[0]+90.  # blind azimuth for first band
phiblind2=phitrig[1]+90.  # blind azimuth for 3rd band
phiblind3=phitrig[2]+90.  # blind azimuth for 3rd band
kN=50
xkblind=[-1.4*kN*np.sin(np.radians(phiblind2)) , 1.4*kN*np.sin(np.radians(phiblind2))]
ykblind=[-1.4*kN*np.cos(np.radians(phiblind2)) , 1.4*kN*np.cos(np.radians(phiblind2))]


X = np.arange(0,nx*dx,dx) # from 0 to (nx-1)*dx with a dx step
Y = np.arange(0,ny*dy,dy)




NameError: name 'img1' is not defined

In [36]:
dir2a=np.degrees(np.arctan2(kx2,ky2))
kn=np.sqrt(kx2**2+ky2**2)*(2*np.pi)+1E-20 #Jusque là les dimmensions sont ok avec Matlab
kncpk=kn/(2*np.pi)*1000


NameError: name 'kx2' is not defined

## Calculation of wave celerity

In [38]:
dir2a=np.degrees(np.arctan2(kx2,ky2))
kn=np.sqrt(kx2**2+ky2**2)*(2*np.pi)+1E-20

dirC1=-100
dirC2=-90

dt=float(dt1[0])
# dt=dt1
J =np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) & (np.degrees(angstd)  < 60) )
J2=np.where((coh > 0.01) & (dir2a < dirC2) & (dir2a > dirC1) )
dispt=np.sqrt(9.81/kn)
Cimg=ang/kn/dt
Cstd=angstd/kn/abs(dt)


NameError: name 'kx2' is not defined

## Fit the phase plane to determine the current components

U and V components of current: U=X[0] and V=X[1]

In [40]:
d2r=np.pi/180
phase_o = abs(dt) * np.sqrt(9.81 * kn)  # phase from linear theory (deep water)
phase_diff=ang-phase_o*np.sign(ang)# phase_diff should be equal to kx.Ux + ky.Uy ...

# FA modifications for later  : manages the possible sign flip : uses the +phase or -phase ... 
# Better trick from Nicolas: work with complex numbers and divide the complex phase by exp(-i*omega*t)  ... 

Clin=np.sqrt(9.81/kn)
philinp= (Clin*kn*dt+pi)%(2*np.pi)-np.pi   # % is the mod operator: np.remainder 
philinm= (-Clin*kn*dt+pi)%(2*pi)-np.pi
Cimg0=(ang-philinm)/kn/dt;
Cimg2=(ang-philinp)/kn/dt;
signchoice=np.where(abs(Cimg2) < abs(Cimg0));
Cimg0[signchoice]=Cimg2[signchoice];
Cimgp=Cimg0;
kfac=1000/(2*np.pi)
J3=np.where( (coh < 0.64) | (kn < 10/kfac) ) #; % | dir2a < 0); % | dir2a > 90);
Cimgp[J3]=numpy.nan;



# k_min_fit = 15 # in cpk
# 17/05/2024
k_min_fit=35  #in cpk

# k_max_fit = 35 # in cpk
# 21/05/2024
k_max_fit=45 # in cpk
std_max = 60*d2r

id_fit = np.where((angstd.flatten()  < std_max)  & (kncpk.flatten() >= k_min_fit) & (kncpk.flatten() <= k_max_fit))[0]  # & phase > 0

angstd1=angstd.flatten()
phase_diff1=phase_diff.flatten()
weight = ntile**2 / (angstd1[id_fit] ** 2)/dt**2
kx1=kx2.flatten()
ky1=ky2.flatten()
K_x_fit = kx1[id_fit]
K_y_fit = ky1[id_fit]
Y = phase_diff1[id_fit] / dt
C=np.diag(weight)
A=(np.column_stack((K_x_fit ,K_y_fit)))*(2*np.pi)  # these are now in rad / m 



# Pondération 

ATC=np.matmul(A.T,C)
D=np.matmul(ATC,A)
#print('A:',A)
DI=np.linalg.inv(D)
X=np.matmul(DI,np.matmul(ATC,Y))
print('U and V components of current :',X)


sigU=np.sqrt(abs(DI))
print('uncertainties for U and V:',sigU[0,0],sigU[1,1])




fig = plt.figure(figsize=(12,8))
ax = fig.add_subplot(111)
ax.scatter(K_x_fit, K_y_fit)
ax.set_title('selected spectral components for fit')



NameError: name 'dt' is not defined